## Quadratic Fields

There are three parts of this assignment:

- Class Numbers of Imaginary Quadratic Fields

- Fundamental Units of Real Quadratic Fields

- Testing the Cohen-Lenstra Heuristics

## Class Numbers (20 points)

Let $D$ be a negative fundamental discriminant.  Compute the class number of the ring of integers of $\mathbf{Q}(\sqrt{-D})$ by counting the number of reduced binary quadratic forms of discriminant $D$.


In [269]:
def reduced_forms(D):
    """Compute the class number of the imaginary quadratic field with fundamental discriminant D"""
    # Check that D is in fact a fundamental discriminant
    assert( D < 0)
    assert (D % 4 == 0 or D % 4 == 1)
    if (D %4 == 0):
        assert Integer(D/4).is_squarefree()
    else:
        assert Integer(D).is_squarefree()
    
    b_0 = D % 2
    B = floor(sqrt(abs(D)/3))
    
    forms = set()
    
    for b in range(b_0, B+2, 2):
        q = (b^2 - D)/4
        al = list()
        for x in range(1, q+1):
            if (q % x == 0):
                al.append(x)
        for a in al:
            if a^2 > q or b > abs(a):
                continue
            f = (a,b,q/a)
            forms.add(f)
            if abs(b) == a or a == q/a:
                continue
            f = (a,-b,q/a)
            forms.add(f)
    return forms

def class_number(D):
    assert( D < 0)
    assert (D % 4 == 0 or D % 4 == 1)
    if (D %4 == 0):
        assert Integer(D/4).is_squarefree()
    else:
        assert Integer(D).is_squarefree()
    return len(reduced_forms(D))

def enum_class_number(R, n):
    tally = 0
    disc = list()
    for x in range(R):
        D = -x
        if (D % 4 == 2) or (D % 4 == 3):
            continue
        if D % 4 == 0: 
            d = Integer(D/4)
            if not d.is_squarefree(): continue
        if D % 4 == 1: 
            d = Integer(D)
            if not d.is_squarefree(): continue
        #print(D)
        if class_number(D) == n:
            tally = tally+1
            disc.append(D)
    return tally, disc
                

In [272]:
print(enum_class_number(3000, 2))
print(enum_class_number(3000, 3))

(20, [-12, -15, -20, -24, -28, -35, -40, -51, -52, -88, -91, -115, -123, -148, -187, -232, -235, -267, -403, -427])
(16, [-23, -31, -59, -83, -107, -139, -211, -283, -307, -331, -379, -499, -547, -643, -883, -907])


Use your function to find all negative fundamental discriminants where the class number is 2, or 3.  You won't be able to prove you've found all of them, but it should be pretty clear you have.

## Fundamental Units (10 points)

Use SageMath's built-in support for continued fractions to find fundamental units for $\mathbf{Q}(\sqrt{107})$ and $\mathbf{Q}(\sqrt{201})$.  Also use SageMath's built-in methods to find the class groups.

In [249]:
def fund_disc(d):
    x = Integer(d).radical()
    if x % 4 == 1:
        if d < 0: return -x
        if d > 0: return x
    if x % 4 == 2 or x % 4 == 3:
        if d < 0: return -4*x
        if d > 0: return 4*x

def fund_unit(d):
    K.<sqrtd> = QuadraticField(d)
    conv = list()
    cand = list()
    f = continued_fraction(sqrtd)
    for i in range(len(f.period())):
        p = f.p(i)
        q = f.q(i)
        c = p^2 - d*q^2
        if fund_disc(d) % 4 == 1:
            if c == 4 or c == -4:
                conv.append(f.convergent(i))
                cand.append(f.p(i) + f.q(i)*sqrtd)
        if c == 1 or c == -1:
            conv.append(f.convergent(i))
            cand.append(f.p(i) + f.q(i)*sqrtd)
    return cand[conv.index(min(conv))]

def gen_disc(n):
    # give the n-th fundamental discriminant
    i = 0
    t = 0
    d_n = 0
    if n > 0: 
        while t < abs(n):
            i = i + 1
            if sage.rings.number_field.number_field.is_fundamental_discriminant(i) == True: t = t + 1; d_n = i
    if n < 0:
        while t < abs(n):
            i = i - 1
            if sage.rings.number_field.number_field.is_fundamental_discriminant(i) == True: t = t + 1; d_n = i
    return d_n
    
        


In [273]:
print(fund_unit(107))
print(fund_unit(201))

93*sqrtd + 962
36332*sqrtd + 515095


## Cohen-Lenstra Heuristics (15 points)

Pick a bound X that is reasonably large.  Collect data about the class numbers and class groups of imaginary quadratic fields with fundamental discriminant $D$ satisfying $-X \leq D < 0$. Check the predictions of the  Cohen-Lenstra Heuristics against your data.  

- The Cohen-Lenstra Heuristics predict that the probability that an odd prime $p$ divides the class number is $1 - (p)_\infty \approx 1/p + 1/p^2$.
- The heuristics predict that around $97.8\%$ of the time the odd part of the class group will be cyclic.  

The basic versions of the Cohen-Lentra heuristics ignore the $2$-part of the class group.  If the prime $2$ behaved like other primes, most of the time the $2$-part of the class group would be trivial or cyclic, and it would be increasingly rare to for the $2$-part to be a product of multiple cyclic groups.
This can be measured by the $2$-torsion in the class group, which reflects the number of cyclic factors whose order is a power of $2$.

- What is the average size of the $2$-torsion in your data?  Does this match the naïve extension of the Cohen-Lenstra heuristics?  

Bonus/Challenge: Can you predict the size of the $2$-torsion just based on the fundamental discriminant?




In [274]:
def sample_IQ_class_groups(n, p):
    i = 0 # iterates from 1 to n
    total_p_torsion_size = 0 # size of 2-torsion submodule
    total_p_torsion = 0 # number of cyclic factors whose order is a multiple of 2
    total = 0 # total number of distinct imaginary quadratic integer rings considered
    total_cyclic_p_torsion = 0 # total number of distinct QIR with cyclic p-torsion
    total_trivial_p_torsion = 0
    while True:
        i = i - 1
        if sage.rings.number_field.number_field.is_fundamental_discriminant(i) == True:
            p_torsion_size = 0 # sets size of p-torsion module to 0
            init = 0;
            p_torsion = 0
            cyclic_p_torsion = 0
            trivial_p_torsion = 1
            total = total + 1
            K = QuadraticField(i)
            G = K.class_group()
            f = G.gens_orders()
            #f = G.elementary_divisors()
            for j in range(len(f)): # for each cyclic prime-power factor
                if f[j] % p == 0: # check if the j-th factor has p-torsion
                    trivial_p_torsion = 0
                    p_torsion = p_torsion + 1 # increment the p-torsion of the group
                    if init == 0: p_torsion_size = f[j].p_primary_part(p); init = 1; cyclic_p_torsion = 1
                    else: p_torsion_size = p_torsion_size*f[j].p_primary_part(p); cyclic_p_torsion = 0
            #print(f)
            #print(trivial_p_torsion)
            #print(cyclic_p_torsion)
            total_p_torsion = total_p_torsion + p_torsion
            total_p_torsion_size = total_p_torsion_size + p_torsion_size
            total_cyclic_p_torsion = total_cyclic_p_torsion + cyclic_p_torsion
            total_trivial_p_torsion = total_trivial_p_torsion + trivial_p_torsion
        if i <= -n: break
            
    print("The average",p,"torsion of a class group (number of factors whose order is divisible by p) is", CC(total_p_torsion/total))
    print("The average size of the",p,"torsion subgroup of a class group is", CC(total_p_torsion_size/total))
    print(total_cyclic_p_torsion, "of", total, "quadratic integer rings (", CC(total_cyclic_p_torsion/total), "percent) tested had cyclic", p, "torsion subgroups")
    print(total_trivial_p_torsion, "of", total, "quadratic integer rings (", CC(total_trivial_p_torsion/total), "percent) tested had trivial", p, "torsion subgroups")

    return total_p_torsion, total_p_torsion_size, total

def divisor_count(n, d):
    divides = 0
    total = 0
    i = 0
    while True:
        i = i - 1
        if sage.rings.number_field.number_field.is_fundamental_discriminant(i) == True: 
            N = class_number(i)
            total = total + 1
            if N % d == 0: divides = divides + 1
        if i <= -n: break
    print(divides, "of", total, "quadratic integer rings (", CC(divides/total), "percent) tested had a class number divisible by", d)
    return divides, total

In [ ]:
divisor_count(3000,5)
sample_IQ_class_groups(3000,5)
divisor_count(3000,2)
sample_IQ_class_groups(3000,2)

164 of 911 quadratic integer rings ( 0.180021953896817 percent) tested had a class number divisible by 5


In [ ]:
# The percentage of the groups tested which had a class number divisible by 2 matches the heuristic (about 75%); but the average 2-torsion is 1.07.